In [2]:
import pandas as pd
import numpy as np

## Intersección de usuarios (**con imágenes**) entre ciudades para domain adaptation

In [3]:
cities = ["Gijon", "Oviedo", "Barcelona", "Warsaw", "Budapest", "Hamburg", "Vienna", "Bucharest", "New York City", "Paris", "Rome", "Madrid", "Berlin", "London", "Istanbul", "Moscow", "St.Petersburg", "Athens"]

In [4]:
cities = ["Gijon"]
cities_data = {}

print(f"{'':10s}\tcategory\titems\treviews\tusers")

for city in cities:
    city = city.lower().replace(" ", "")
    city_path = f"out/{city}"

    cities_data[city] = {"restaurants":{}, "pois":{}}

    for category in ["restaurants", "pois"]:
        category_path = f"{city_path}/{category}"

        item_path = f"{category_path}/items.pkl"
        review_path = f"{category_path}/reviews.pkl"
        user_path = f"{category_path}/users.pkl"

        items = pd.read_pickle(item_path)
        reviews = pd.read_pickle(review_path)
        users = pd.read_pickle(user_path)

        reviews["n_images"] = reviews["images"].apply(lambda x: len(x))
        users_with_images = reviews.loc[reviews.n_images>0]["userId"].unique()

        cities_data[city][category] = {"items":items, "reviews":reviews, "users":users, "users_images":users_with_images}
        
        print(f"{city:10s}\t{category:15s}\t{len(items)}\t{len(reviews)}\t{len(users)}")

          	category	items	reviews	users
gijon     	restaurants    	849	82145	42384
gijon     	pois           	312	16868	9856


In [9]:
city_names = "\t".join(cities_data.keys())
print(f'city\t{city_names}')
      
for c_urs in cities_data.keys():
  print(c_urs, end="\t")
  for o_urs in cities_data.keys():
    c_data = set(cities_data[c_urs]["restaurants"]["users_images"])
    o_data = set(cities_data[o_urs]["restaurants"]["users_images"])
    print(len(c_data.intersection(o_data)), end="\t")
  print()

city	gijon	oviedo	barcelona	warsaw	budapest	hamburg	vienna	bucharest	newyorkcity	paris	rome	madrid	berlin	london	istanbul	moscow	st.petersburg	athens
gijon	10113	1414	667	52	157	26	131	32	183	252	407	1917	165	266	72	26	26	80	
oviedo	1414	9476	668	70	192	33	142	34	195	263	432	1740	187	290	70	36	34	98	
barcelona	667	668	178938	1299	3889	1105	3500	867	5251	9609	10684	11389	3961	8931	2090	1434	1177	2253	
warsaw	52	70	1299	25988	1422	308	1093	521	734	1238	1661	1082	1243	1507	490	507	416	704	
budapest	157	192	3889	1422	81719	674	5253	1102	2308	3978	5365	3038	3028	4834	1487	1209	1067	1568	
hamburg	26	33	1105	308	674	20043	1169	161	776	1153	1019	673	2402	1356	326	239	203	336	
vienna	131	142	3500	1093	5253	1169	55469	800	2328	4126	4874	2780	3349	4309	1360	1151	963	1385	
bucharest	32	34	867	521	1102	161	800	20430	496	915	1411	754	651	1091	572	260	220	632	
newyorkcity	183	195	5251	734	2308	776	2328	496	98468	7478	6507	4105	2794	9008	1337	808	709	1514	
paris	252	263	9609	1238	3978	1153	4126	915	74

In [11]:
city_names = "\t".join(cities_data.keys())
print(f'city\t{city_names}')
      
for c_urs in cities_data.keys():
  print(c_urs, end="\t")
  for o_urs in cities_data.keys():
    c_data = set(cities_data[c_urs]["restaurants"]["reviews"]["userId"].unique())
    o_data = set(cities_data[o_urs]["pois"]["reviews"]["userId"].unique())
    print(len(c_data.intersection(o_data)), end="\t")
  print()

city	gijon	oviedo	barcelona	warsaw	budapest	hamburg	vienna	bucharest	newyorkcity	paris	rome	madrid	berlin	london	istanbul	moscow	st.petersburg	athens
gijon	4859	1883	860	162	575	132	321	94	615	723	687	3179	819	1168	257	101	121	185	
oviedo	1775	5108	1034	169	621	134	394	97	696	867	816	3257	931	1223	334	120	161	232	
barcelona	908	1180	100320	3734	14429	3802	10403	2330	19826	27365	22569	26667	18632	33405	9214	3962	4753	7326	
warsaw	83	100	2559	21909	4550	1023	3014	1363	2634	3252	3005	2520	4560	4944	1898	1487	1370	1625	
budapest	235	340	8876	4383	89333	2535	14509	3144	9520	12071	11140	8105	13759	18173	6378	3493	3992	4776	
hamburg	55	80	2394	952	2409	24454	3482	478	3036	3251	2561	1875	8347	4956	1493	781	955	1025	
vienna	203	288	8139	3298	17387	4543	62571	2217	9691	13357	11372	7906	14677	16684	5994	3770	4105	4388	
bucharest	61	79	1826	1474	3514	612	2246	16397	1711	2370	2487	1788	2608	3448	2021	888	888	1596	
newyorkcity	271	414	16199	3117	12104	3191	10102	1816	169805	32558	23422	14375	16559	4

In [ ]:
def fotos_por_restaurante(reviews, city):
  data = reviews.groupby("restaurantId").n_img.sum().reset_index()
  histo = sbs.histplot(data, x="n_img", kde=False, bins=range(1,22,1));
  histo.set_xticks(np.arange(1,21,1)+.5); # Etiquetas del eje X
  labels = list(map(str,range(1,20)))+["≥20"]
  histo.set_xticklabels(labels)
  histo.set_xlabel("Número de fotos")
  histo.set_ylabel("Número de restaurantes")
  plt.tight_layout()
  histo.figure.savefig(f"{city}_hist_fotos_pr_rst.pdf")
  plt.clf()
  
def fotos_por_review(reviews, city):
  return reviews
def fotos_por_usuario(reviews, city):
  return reviews
def reviews_por_restaurante(reviews, city):
  data = reviews.groupby("restaurantId").reviewId.count().reset_index()
  data.loc[data.reviewId>=20, "reviewId"]=20

  histo = sbs.histplot(data, x="reviewId", kde=False, bins=range(1,22,1));
  histo.set_xticks(np.arange(1,21,1)+.5); # Etiquetas del eje X
  labels = list(map(str,range(1,20)))+["≥20"]
  histo.set_xticklabels(labels)
  histo.set_xlabel("Número de reseñas")
  histo.set_ylabel("Número de restaurantes")
  plt.tight_layout()
  histo.figure.savefig(f"{city}_hist_rvws_pr_rst.pdf")

def reviews_por_usuario(reviews, city):
  return reviews

In [ ]:
cities = ["gijon", "barcelona", "paris", "madrid", "newyorkcity", "london"]
# cities = ["gijon"]

base_path = "/content/drive/Shareddrives/Media"

for city in cities:
  data_path = f"/content/drive/Shareddrives/DataSets/TripAdvisor/{city}/"
  reviews = pd.read_pickle(data_path+"reviews.pkl")

  # reviews["n_img"] = reviews.images.apply(lambda x: len(x))
  restaurants = pd.read_pickle(data_path+"restaurants.pkl")
  users = pd.read_pickle(data_path+"users.pkl")
  
  # dates = pd.to_datetime(reviews.date, format="%d de %B de %Y" )
  # print(dates.min(), dates.max())

  # n_images  = reviews.n_img.sum()
  # dt = [city, len(reviews), len(users), len(restaurants), n_images]
  # print("\\"+" & ".join(map(str, dt))+" \\\\")

  reviews = reviews[["date", "images", "language", "rating", "restaurantId", "reviewId", "text", "title", "url", "userId"]]
  users = users[["id", "name", "location"]]
  restaurants = restaurants[["id", "name", "city", "priceInterval", "url", "rating"]]

  reviews.iloc[:50].to_csv(f"{base_path}/{city}_revs.csv", encoding="utf-8")
  users.iloc[:50].to_csv(f"{base_path}/{city}_usrs.csv", encoding="utf-8")
  restaurants.iloc[:50].to_csv(f"{base_path}/{city}_rsts.csv", encoding="utf-8")

In [ ]:
reviews["text"] = reviews.text.str.lower().str.split(" ")

In [ ]:
reviews.text.str.lower().str.contains("definitivas").value_counts()

In [ ]:
cities = {"gijon":0.3}#, "barcelona":1.6, "madrid":2.1, "newyorkcity":8.3, "paris":8.9, "london":3.2} # Ciudades con su población
stats = []

for city in cities.keys():
  data_path = f"/content/drive/Shareddrives/DataSets/TripAdvisor/{city}/"

  reviews = pd.read_pickle(data_path+"reviews.pkl")
  # restaurants = pd.read_pickle(data_path+"restaurants.pkl")
  # users = pd.read_pickle(data_path+"users.pkl")

  reviews["n_img"] = reviews.images.apply(lambda x: len(x))

  n_reviews = len(reviews.reviewId.unique())
  n_images  = reviews.n_img.sum()
  n_users = len(reviews.userId.unique())
  n_rest = len(reviews.restaurantId.unique())

  stats.append((f"\{city}", cities[city], n_reviews, n_images, n_users, n_rest))

  fotos_por_restaurante(reviews, data_path+city)
  reviews_por_restaurante(reviews, data_path+city)

stats = pd.DataFrame(stats, columns=["Conjunto", "Población", "Reseñas", "Imágenes", "Usuarios", "Restaurantes"])
print(stats.to_latex(index=False, escape=False))

In [ ]:
data = reviews.groupby("restaurantId").n_img.sum().reset_index()
data.loc[data.n_img>=20, "n_img"]=20

histo = sbs.histplot(data, x="n_img", kde=False, bins=range(1,22,1));
histo.set_xticks(np.arange(1,21,1)+.5); # Etiquetas del eje X
labels = list(map(str,range(1,20)))+["≥20"]
histo.set_xticklabels(labels)
histo.set_xlabel("Número de fotos")
histo.set_ylabel("Número de restaurantes")
plt.tight_layout()


In [ ]:
# Datos para Bea (con menos columnas)
data_sm = reviews[["reviewId", "userId", "restaurantId", "date", "images", "rating", "url"]]
data_sm.to_pickle(data_path+"reviews_sm.pkl")


## Número de restaurantes con datos de precio


In [3]:

cities = ["Istanbul", "Moscow", "St. Petersburg", "Athens","Rome"]
cities = ["Gijon", "Barcelona", "Warsaw", "Budapest", "Hamburg", "Vienna", "Bucharest", "New York City", "Paris", "Madrid", "Berlin", "London", "Porto", "Lisbon"]



res = []

for c in cities:
    c = c.lower().replace(" ", "")
    reviews_p_path = f"out/{c}/restaurants/items.pkl"
    reviews = pd.read_pickle(reviews_p_path)
    a,b = np.unique(reviews["priceInterval"], return_counts=True)
    city_data = dict(zip(a,b))
    city_data["city"] = c
    res.append(pd.DataFrame(city_data, index=[0]).values.tolist()[0])
    
pd.DataFrame(res, columns=[list(city_data.keys())])[list(city_data.keys())[::-1]]

FileNotFoundError: [Errno 2] No such file or directory: 'out/istanbul/restaurants/items.pkl'